## 3.1.9 Realistic simulation setup
Continuation of the previous script, now with multiple object coordinates

### Plot spot with pixel diameter
- ~~create a mesh with pixel size distance corners~~
- ~~create a spot centered in the middle of the plane~~
- ~~Pass the spot through the mesh~~ 
- ~~Each pixel that has the spot goes to a micro mesh~~
- ~~Each micropixel that has the spot sum.~~
- ~~Consider the case where the spot is inbetween two pixels~~
    - ~~Expand the mesh by 1 in each direction~~
- ~~Consider the case where the spot is inbetween 4 pixels~~
- Consider the case where:
    - ~~spot is double the size centered inbetween  4 pixels~~
    ~~- spot is centered in the middle of a pixel~~
- ~~Approach where the inner and outter circle parts are taken into consideration~~
    ~~- Stereo setup~~
- ~~centroid calculation~~
    - Test with multiple points

### import  libraries

In [550]:
vscode = 1

In [551]:
if(vscode == 0):
    # for vscode
    # for jupyter notebook
    from mpl_toolkits.mplot3d import axes3d
    import matplotlib.pyplot as plt

    %matplotlib notebook
elif(vscode == 1):
    %matplotlib qt
    

In [552]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 
from typing import Sequence
from calib_lib import *
from matplotlib.collections import PatchCollection      # pathcoletion for multiple patches
import matplotlib.cm as cm                              # for colormaps
from matplotlib.patches import Rectangle                # for rectangle representation

DECIMALS = 2            # how many decimal places to use in print


In [553]:
F = 16                                       # focal length( in mm )
gridSize = 1024
image_size = np.array([1936,1216])               # sensor size(in mm)
#image_size = np.array([11.345,7.126])
pixel_width = 5.86e-3                       # pixel size in mm
PX= image_size[0]/2.0                       # principal point x-coordinate
PY= image_size[1]/2.0                       # principal point y-coordinate
IMAGE_HEIGTH = image_size[1]
IMAGE_WIDTH = image_size[0]
THETA_X = 0                                 # roll angle
THETA_Y = 0                                 # pitch angle
THETA_Z = 0                                 # yaw angle

# camera Right
THETA_X_R = 0                                 # roll angle
THETA_Y_R = 0                                 # pitch angle
THETA_Z_R= 0                                 # yaw angle
# camera Left
THETA_X_L = 0                                 # roll angle
THETA_Y_L = 0                                 # pitch angle
THETA_Z_L= 0                                 # yaw angle

C_L = np.array([0,0,0])                     # camera centre
C_R = np.array([500,0,0])

chess_dimx,chess_dimy = (13,9)
chess_sq_size = 44

## 3D spot
 - Create function able of creating 4 points with the centroid being the previous infinitesimal coordinate
    - Create the normal chessboard with the centroid being the previous coordinates
    - Create a pattern of 4 points around the centroid

In [554]:
def create_chessboard_4points(patternsize_x,patternsize_y,dim):
    """
    Creates chessboard with pattern dimensions for pixelization - create 4 points around centroid
    
    Parameters
    ----------
    patternsize_x : int
        size x of the chessboard 
    patternsize_y : int
        size y of the chessboard
    dim:    int
        distance between corners in mm

    Returns
    -------
    world_pts_arr :  : np.array
        chessboard points in 3D
    """
    pattern_size_x,pattern_size_y = (patternsize_x,patternsize_y)

    X = np.linspace(-pattern_size_x/2, pattern_size_x/2,pattern_size_x + 1)*dim
    Y = np.linspace(-pattern_size_y/2, pattern_size_y/2,pattern_size_y + 1)*dim

    zdata = np.zeros((patternsize_x + 1,patternsize_y + 1))

    xdata, ydata = np.meshgrid(X, Y)

    xdata_ = xdata.flatten()
    ydata_ = ydata.flatten()
    zdata_ = zdata.flatten()
    onedata = np.ones((zdata_.shape[0])).flatten()
    #
    spots = []
    for i in range(xdata_.shape[0]):
        spots.append(np.array([ [xdata_[i]-5,ydata_[i],zdata_[i],onedata[i]],
                                [xdata_[i],ydata_[i]+5,zdata_[i],onedata[i]],
                                [xdata_[i]+5,ydata_[i],zdata_[i],onedata[i]],
                                [xdata_[i],ydata_[i]-5,zdata_[i],onedata[i]]]))

    spots_array = np.array(spots)

    return spots_array


In [555]:
def get_image_points(rand_points,px,py,thetax = 0, thetay = 0, thetaz = 0,trans_x = 0,trans_y = 0,trans_z = 0,F = 3.0,mx = 1,my = 1):
    """
    from the world points, obtains the correspondent image coordinate

    Parameters
    ----------
    rand_points:  np.array
        world points
    F : float
        focal length
    px : float
        principal point in the x diretion
    py : float
        principal point in the y diretion
    trans_x : float
       translation on the x axis
    trans_y : float
       translation on the y axis
    trans_z : float
        translation on the z axis
    thetax : float
        angle in the x axis in radians
    thetay : float
        angle in the y axis in radians
    thetaz : float
        angle in the z axis in radians
    mx : float
        pixel density in the x diretion
    my : float
        pixel density in the y diretion
    Returns
    -------
    x_arr:    np.array
        image coordinates
    rand_points : np.array
        world coordinates
    """
    K = get_calibration_matrix(F, px=px, py=py,mx=mx,my=my)                                             # calibration matrix
    P, E = get_projection_matrix(F, px=px,py =py,theta_x = thetax,theta_y = thetay,theta_z = thetaz,tx = trans_x,ty = trans_y, tz = trans_z,mx = mx,my =my)                        # projection matrix
    #print("\nCalibration matrix (K):\n", K)
    #print("\nProjection matrix (P):\n", P)

    x = []

    for i in range(rand_points.shape[0]):
        Xh = to_homogeneous(rand_points[i,:])
        xh = P @ Xh
        Xx = to_inhomogeneous(xh)
        x.append(Xx)

    x_arr = np.array(x)

    return x_arr, rand_points, E, K, P

In [556]:
def calib(world_pts_arr,x_zhang_R,x_zhang_L):    
    ret_R, mtx_R, dist_R, rvecs_R, tvecs_R = cv2.calibrateCamera(world_pts_arr, x_zhang_R, (image_size[0],image_size[1]), None, None)
    ret_L, mtx_L, dist_L, rvecs_L, tvecs_L = cv2.calibrateCamera(world_pts_arr, x_zhang_L, (image_size[0],image_size[1]), None, None)

    flags = 0
    flags |= cv2.CALIB_FIX_INTRINSIC
    # Here we fix the intrinsic camara matrixes so that only Rot, Trns, Emat and Fmat are calculated.
    # Hence intrinsic parameters are the same 

    criteria_stereo= (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


    # This step is performed to transformation between the two cameras and calculate Essential and Fundamenatl matrix
    retS, new_mtxL, distL, new_mtxR, distR, Rot, Trns, Emat, Fmat = cv2.stereoCalibrate(world_pts_arr, x_zhang_L, x_zhang_R, mtx_L, dist_L, mtx_R, dist_R, (image_size[0],image_size[1]), criteria_stereo, flags)

    return ret_R,mtx_R,ret_L,mtx_L,retS,new_mtxL,new_mtxR,Rot,Trns,Emat,Fmat

In [557]:
def create_4_point_chess(xrange,yrange,zrange,chess_dimx = chess_dimx,chess_dimy = chess_dimy,chess_sq_size = chess_sq_size):

    translst= []
    spots = []

    for z in zrange:
        for x in xrange:
            for y in yrange:
                translst.append([x,y,z])
                
    # create the normal chessboard
    chess_pts = []
    world_pts = create_chessboard_4points(chess_dimx,chess_dimy,chess_sq_size)
    rotangles = [[0,0,0]]
    for i in range(len(translst)):
        for j in range(len(rotangles)):    
            chess_pts.append(get_chessboard_rot_trans(world_pts,rotangles[j][0],rotangles[j][1],rotangles[j][2],translst[i][0],translst[i][1],translst[i][2]))

    chess_pts_arr = np.array(chess_pts)

    pattern_size_x,pattern_size_y = (chess_dimx,chess_dimy)

    X_pattern = np.linspace(0, pattern_size_x,pattern_size_x + 1)*chess_sq_size
    Y_pattern = np.linspace(0, pattern_size_y,pattern_size_y + 1)*chess_sq_size
    zdata = np.zeros((pattern_size_x + 1,pattern_size_y + 1))
    xdata, ydata = np.meshgrid(X_pattern, Y_pattern)

    xdata_ = xdata.flatten()
    ydata_ = ydata.flatten()
    zdata_ = zdata.flatten()

    spots = []
    for i in range(xdata_.shape[0]):
        spots.append(np.array([ [xdata_[i]-5,ydata_[i],zdata_[i]],
                                [xdata_[i],ydata_[i]+5,zdata_[i]],
                                [xdata_[i]+5,ydata_[i],zdata_[i]],
                                [xdata_[i],ydata_[i]-5,zdata_[i]]]))

    spots_array = np.array(spots)

    # homogeneous coordinates
    world_pts =([xdata_,ydata_,zdata_])
    world_pts_ = np.array(world_pts).T

    world_pts_arr = np.zeros((chess_pts_arr.shape[0],(pattern_size_x+1)*(pattern_size_y+1),4,3),np.float32)
    for i in range(chess_pts_arr.shape[0]):
        world_pts_arr[i,:,:,:] = spots_array

    world_pts_arr = world_pts_arr.reshape((chess_pts_arr.shape[0],1,(pattern_size_x+1)*(pattern_size_y+1)*4,3))

    x_zhang_R = np.zeros((chess_pts_arr.shape[0],(pattern_size_x+1)*(pattern_size_y+1)*4,2),np.float32)
    x_zhang_L = np.zeros((chess_pts_arr.shape[0],(pattern_size_x+1)*(pattern_size_y+1)*4,2),np.float32)
    x_arr_L = np.zeros(((pattern_size_x+1)*(pattern_size_y+1),4,2))
    x_arr_R = np.zeros(((pattern_size_x+1)*(pattern_size_y+1),4,2))
    for i in range(chess_pts_arr.shape[0]):
        for j in range(chess_pts_arr.shape[1]):
            x_R, X_R, E_R, K_R, P_R = get_image_points(chess_pts_arr[i,j,:,:],PX,PY,thetax= 0,thetay = 0,thetaz = 0,trans_x= -C_R[0],trans_y= -C_R[1],trans_z= -C_R[2],F = F,mx = (1/pixel_width),my =(1/pixel_width))
            x_L, X_L, E_L, K_L, P_L = get_image_points(chess_pts_arr[i,j,:,:],PX,PY,thetax= 0,thetay = 0,thetaz = 0,trans_x= -C_L[0],trans_y= -C_L[1],trans_z= -C_L[2],F = F,mx = (1/pixel_width),my =(1/pixel_width))
            
            x_arr_L[j,:,:] = x_L
            x_arr_R[j,:,:] = x_R

        
        x_zhang_R[i,:,:] = x_arr_R.reshape((pattern_size_x+1)*(pattern_size_y+1)*4,2)
        x_zhang_L[i,:,:] = x_arr_L.reshape((pattern_size_x+1)*(pattern_size_y+1)*4,2)

    
    x_zhang_R = x_zhang_R.reshape((chess_pts_arr.shape[0],(pattern_size_x+1)*(pattern_size_y+1),4,2))
    x_zhang_L = x_zhang_L.reshape((chess_pts_arr.shape[0],(pattern_size_x+1)*(pattern_size_y+1),4,2))



    print("World points: ",world_pts_arr.shape)
    print("Image points: ",x_zhang_L.shape)

    return world_pts_arr,x_zhang_R,x_zhang_L,P_R,P_L      
       
xrange_5 = np.linspace(-6000,6000,6)
yrange_5 = np.linspace(-4000,4000,6)
zrange_5 = ([20000])


world_pts_arr,x_zhang_R,x_zhang_L,P_R,P_L = create_4_point_chess(xrange_5,yrange_5,zrange_5)


#ret_R,mtx_R,ret_L,mtx_L,retS,new_mtxL,new_mtxR,Rot,Trns,Emat,Fmat = calib(world_pts_arr,x_zhang_R,x_zhang_L)



World points:  (36, 1, 560, 3)
Image points:  (36, 140, 4, 2)


In [558]:
world_pts_arr

array([[[[ -5.,   0.,   0.],
         [  0.,   5.,   0.],
         [  5.,   0.,   0.],
         ...,
         [572., 401.,   0.],
         [577., 396.,   0.],
         [572., 391.,   0.]]],


       [[[ -5.,   0.,   0.],
         [  0.,   5.,   0.],
         [  5.,   0.,   0.],
         ...,
         [572., 401.,   0.],
         [577., 396.,   0.],
         [572., 391.,   0.]]],


       [[[ -5.,   0.,   0.],
         [  0.,   5.,   0.],
         [  5.,   0.,   0.],
         ...,
         [572., 401.,   0.],
         [577., 396.,   0.],
         [572., 391.,   0.]]],


       ...,


       [[[ -5.,   0.,   0.],
         [  0.,   5.,   0.],
         [  5.,   0.,   0.],
         ...,
         [572., 401.,   0.],
         [577., 396.,   0.],
         [572., 391.,   0.]]],


       [[[ -5.,   0.,   0.],
         [  0.,   5.,   0.],
         [  5.,   0.,   0.],
         ...,
         [572., 401.,   0.],
         [577., 396.,   0.],
         [572., 391.,   0.]]],


       [[[ -5.,   0.,   0.

In [559]:
""" fig = plt.figure()
ax = fig.add_subplot(121)
ax.plot(x_zhang_L[:,:,0],x_zhang_L[:,:,1], color = 'r',ls = "None", marker = "*")
ax.grid()
ax.set_xlim([0,image_size[0]])
ax.set_ylim([0,image_size[1]])
ax.set_aspect(1)

ax1 = fig.add_subplot(122)
ax1.plot(x_zhang_R[:,:,0],x_zhang_R[:,:,1], color = 'r',ls = "None", marker = "*")
ax1.grid()
ax1.set_xlim([0,image_size[0]])
ax1.set_ylim([0,image_size[1]])
ax1.set_aspect(1)
plt.show() """

' fig = plt.figure()\nax = fig.add_subplot(121)\nax.plot(x_zhang_L[:,:,0],x_zhang_L[:,:,1], color = \'r\',ls = "None", marker = "*")\nax.grid()\nax.set_xlim([0,image_size[0]])\nax.set_ylim([0,image_size[1]])\nax.set_aspect(1)\n\nax1 = fig.add_subplot(122)\nax1.plot(x_zhang_R[:,:,0],x_zhang_R[:,:,1], color = \'r\',ls = "None", marker = "*")\nax1.grid()\nax1.set_xlim([0,image_size[0]])\nax1.set_ylim([0,image_size[1]])\nax1.set_aspect(1)\nplt.show() '

### 2D correspondance

In [560]:
def findcircle(x_arr):
    """
    Based on the image coordinates, and knowing it forms a square/rectangle, finds the centroid and radius

    Parameters
    ----------
    x_arr:  np.array
        image coordinates
    Returns
    -------
    cx0:    np.float
        centroid x
    cy0:    np.float
        centroid y    
    r0 : np.float
        circle radius
    """  
    
    Ax  = x_arr[0,0]
    Ay  = x_arr[0,1]
    Bx  = x_arr[1,0]
    By  = x_arr[1,1]
    Cx  = x_arr[2,0]
    Cy  = x_arr[2,1]
    Dx  = x_arr[3,0]
    Dy  = x_arr[3,1]

    r0 = (Cx-Ax)/2
    cx0 = (Ax+Cx)/2
    cy0 = (By+Dy)/2
    return cx0,cy0,r0

def createcircle(x_arr,nrpoints = 100):
    """
    create the circle, using the function findcircle()

    Parameters
    ----------
    x_arr:  np.array
        image coordinates
    nrpoints: integer
        number of points that compose the circle
    Returns
    -------
    circle_x:    np.array
        points that define the circle in the cx0 coordinate
    circle_y:    np.array
        points that define the circle in the cy0 coordinate
    cx0:    np.float
        centroid x
    cy0:    np.float
        centroid y    
    r0 : np.float
        circle radius    
    """    

    cx0,cy0,r0 = findcircle(x_arr)

    theta = np.linspace(0, 2*np.pi, nrpoints)
    x = r0*np.cos(theta)
    y = r0*np.sin(theta)

    circle_x = x+cx0
    circle_y = y+cy0


    return circle_x, circle_y,cx0,cy0,r0

## Plot of the spot in the image plane

In [561]:
# Generate pixel grid
pixelGridX  = (np.arange(0,image_size[0])) * pixel_width
pixelGridY  = (np.arange(0,image_size[1])) * pixel_width
pixelGridXX, pixelGridYY = np.meshgrid(pixelGridX,pixelGridY)
# magnification 
zoom_1 = 5
zoom_2 = 3
### Left image

fig = plt.figure(figsize=(20,20))
# sup title
fig.suptitle("Left and Right image plane coordinates")

ax_fig_L = fig.add_subplot(121)
ax_fig_L.plot(pixelGridXX,pixelGridYY,'b.-', label = 'Original')
ax_fig_L.plot(pixelGridXX.T,pixelGridYY.T, 'r.-', label = 'Original')

cx0_L = np.zeros((x_zhang_L.shape[0],x_zhang_L.shape[1],1))
cy0_L = np.zeros((x_zhang_L.shape[0],x_zhang_L.shape[1],1))
r0_L = np.zeros((x_zhang_L.shape[0],x_zhang_L.shape[1],1))
cx0_R = np.zeros((x_zhang_R.shape[0],x_zhang_R.shape[1],1))
cy0_R = np.zeros((x_zhang_R.shape[0],x_zhang_R.shape[1],1))
r0_R = np.zeros((x_zhang_R.shape[0],x_zhang_R.shape[1],1))

#
x0_L = np.zeros((x_zhang_L.shape[0],x_zhang_L.shape[1],gridSize))
y0_L = np.zeros((x_zhang_L.shape[0],x_zhang_L.shape[1],gridSize))
x0_R = np.zeros((x_zhang_R.shape[0],x_zhang_R.shape[1],gridSize))
y0_R = np.zeros((x_zhang_R.shape[0],x_zhang_R.shape[1],gridSize))

for i in range(x_zhang_L.shape[0]):
    for j in range(x_zhang_L.shape[1]):

        #circle definition 
        # multiply by pixel_width to get the results in mm, not in pixels
        x0_L[i,j,:],y0_L[i,j,:],cx0_L[i,j,:],cy0_L[i,j,:],r0_L[i,j,:] = createcircle(x_zhang_L[i,j,:]*pixel_width,gridSize)
        cc_L = plt.Circle(( cx0_L[i,j,:] ,cy0_L[i,j,:] ), r0_L[i,j,:])

        ax_fig_L.add_artist( cc_L ) 
        ax_fig_L.set_aspect( 1 ) 
        #
        ax_fig_L.plot(cx0_L[i,j,:],cy0_L[i,j,:], color = 'y',ls = "None", marker = ".")
        ax_fig_L.plot(x_zhang_L[i,j,:,0]*pixel_width,x_zhang_L[i,j,:,1]*pixel_width, color = 'y',ls = "None", marker = ".")
        #limits
        ax_fig_L.set_xlim([np.min(cx0_L)-(zoom_1*pixel_width),np.max(cx0_L)+(zoom_1*pixel_width)])
        ax_fig_L.set_ylim([np.min(cy0_L)-(zoom_1*pixel_width),np.max(cy0_L)+(zoom_1*pixel_width)])

ax_fig_R = fig.add_subplot(122)
ax_fig_R.plot(pixelGridXX,pixelGridYY,'b.-', label = 'Original')
ax_fig_R.plot(pixelGridXX.T,pixelGridYY.T, 'r.-', label = 'Original')

for i in range(x_zhang_R.shape[0]):
    for j in range(x_zhang_R.shape[1]):

        #circle definition 
        # multiply by pixel_width to get the results in mm, not in pixels
        x0_R[i,j,:],y0_R[i,j,:],cx0_R[i,j,:],cy0_R[i,j,:],r0_R[i,j,:] = createcircle(x_zhang_R[i,j,:]*pixel_width,gridSize)
        cc_R = plt.Circle(( cx0_R[i,j,:] ,cy0_R[i,j,:] ), r0_R[i,j,:])
        ax_fig_R.add_artist( cc_R ) 
        ax_fig_R.set_aspect( 1 ) 
        #
        ax_fig_R.plot(cx0_R[i,j,:],cy0_R[i,j,:], color = 'y',ls = "None", marker = ".")
        ax_fig_R.plot(x_zhang_R[i,j,:,0]*pixel_width,x_zhang_R[i,j,:,1]*pixel_width, color = 'y',ls = "None", marker = ".")
        #limits
        ax_fig_R.set_xlim([np.min(cx0_R)-(zoom_1*pixel_width),np.max(cx0_R)+(zoom_1*pixel_width)])
        ax_fig_R.set_ylim([np.min(cy0_R)-(zoom_1*pixel_width),np.max(cy0_R)+(zoom_1*pixel_width)])

plt.show()

# Left Spot 

In [562]:
def spot_center_coord(cx0,cy0,pixelGridX,pixelGridY):
    cx0_max = np.where((cx0 < pixelGridX))
    cx0_min = np.where((cx0 > pixelGridX))
    #print(cx0_max[0][0])
    #print(cx0_min[0][-1])
    cy0_max = np.where((cy0 < pixelGridY))
    cy0_min = np.where((cy0 > pixelGridY))
    #print(cy0_max[0][0])
    #print(cy0_min[0][-1])

    return cx0_max[0][0],cx0_min[0][-1],cy0_max[0][0],cy0_min[0][-1]

In [563]:
def pixel_cover_finder(gridSize,cx0max,cy0max,cx0,cy0,r):
    # array for percentage storage
    pixel_covered_percent = np.zeros((1,))

    xgrid = np.linspace(0,1,gridSize)*pixel_width
    ygrid = np.linspace(0,1,gridSize)*pixel_width

    x_pixel = cx0max+xgrid
    y_pixel = cy0max+ygrid

    xx_pixel, yy_pixel = np.meshgrid(x_pixel,y_pixel)

    circ_total = np.sqrt((xx_pixel-cx0)**2 + (yy_pixel-cy0)**2) <= r
    circ = np.where(circ_total == True)

    pixel_covered = circ[0].shape
    totalpixel_area = gridSize**2
    pixel_covered_percent = (pixel_covered[0]/totalpixel_area)*100
    #print("pixel cover percentage: " + str(pixel_covered_percent) + " %")

    return pixel_covered_percent

Create an array that has a list for each point

In [564]:
points_max_min_L =np.zeros((x0_L.shape[0],x0_L.shape[1],x0_L.shape[2],2))

for i in range(x0_L.shape[0]):
    for j in range(x0_L.shape[1]):
        for k in range(x0_L.shape[2]):

            cx0max,cx0min,cy0max,cy0min = spot_center_coord(x0_L[i,j,k],y0_L[i,j,k],pixelGridX,pixelGridY)

            # max and min points in (x,y) coordinates
            cx0min_pxl_L = (cx0min*pixel_width)
            cx0max_pxl_L = (cx0max*pixel_width)
            cy0min_pxl_L = (cy0min*pixel_width)
            cy0max_pxl_L = (cy0max*pixel_width)

            #array max and min points
            points_max_min_L[i,j,k,:] = (([cx0min_pxl_L,cy0min_pxl_L]))



KeyboardInterrupt: 

### Remove outter circle duplicates

In [ ]:
points_deduplt_L = list()
points_outter_L = []
for i in range(points_max_min_L.shape[0]):
    points_outter_L_list = []
    for j in range(points_max_min_L.shape[1]):
        points_deduplt_L = []
        points_max_min_dupl_L = points_max_min_L[i,j,:,:].tolist()

        [points_deduplt_L.append(item) for item in points_max_min_dupl_L if item not in points_deduplt_L]

        points_outter_L_list.append(points_deduplt_L)
        
    points_outter_L.append(points_outter_L_list)
np.shape(points_outter_L)

In [ ]:
cx0_L.shape

## Inner circle corners

In [ ]:
points_inner_L = []
for i in range(cx0_L.shape[0]):
    points_inner_L_list = []
    for j in range(cx0_L.shape[1]):

        circ_L = np.sqrt((pixelGridXX-cx0_L[i,j,:])**2 + (pixelGridYY-cy0_L[i,j,:])**2) <= r0_L[i,j,:]
        circ_corners_L = np.where(circ_L == True)
        circ_corners_L = np.array(circ_corners_L)

        # transpose and obtain the values in mm
        circ_corners_arr_L = (circ_corners_L*pixel_width).T
        # swap the coordinates(for better readability)
        circ_corners_arr_L = np.array([circ_corners_arr_L[:,1],circ_corners_arr_L[:,0]]).T
        points_inner_L_list.append(circ_corners_arr_L.tolist())

    points_inner_L.append(points_inner_L_list)
#points_inner_L


In [ ]:
points_total_L = []
for i in range(cx0_L.shape[0]):
    points_total_L_list = []
    for j in range(cx0_L.shape[1]):
        points_total_L_list.append(points_inner_L[i][j]+points_outter_L[i][j])
    points_total_L.append(points_total_L_list)

In [ ]:
np.shape(points_total_L)


In [ ]:
points_L_list = points_total_L[0][0][:][:]
points_L_list

In [ ]:
points_L_list = points_total_L[0][0][:][:]
points_deduplt_L = []
[points_deduplt_L.append(item) for item in points_L_list if item not in points_deduplt_L]
points_deduplt_L

### Remove outter and inner circle duplicates

In [ ]:
points_L = []
for i in range(np.shape(points_total_L)[0]):
    points_total_L_list = []
    for j in range(np.shape(points_total_L)[1]):
        points_L_list = []
        points_L_lst = points_total_L[i][j][:][:]

        [points_L_list.append(item) for item in points_L_lst if item not in points_L_list]

        points_total_L_list.append(points_L_list)
        
    points_L.append(points_total_L_list)
np.shape(points_L)

# Right spot

In [ ]:
points_max_min_R =np.zeros((x0_R.shape[0],x0_R.shape[1],x0_R.shape[2],2))

for i in range(x0_R.shape[0]):
    for j in range(x0_R.shape[1]):
        for k in range(x0_R.shape[2]):

            cx0max,cx0min,cy0max,cy0min = spot_center_coord(x0_R[i,j,k],y0_R[i,j,k],pixelGridX,pixelGridY)

            # max and min points in (x,y) coordinates
            cx0min_pxl_R = (cx0min*pixel_width)
            cx0max_pxl_R = (cx0max*pixel_width)
            cy0min_pxl_R = (cy0min*pixel_width)
            cy0max_pxl_R = (cy0max*pixel_width)

            #array max and min points
            points_max_min_R[i,j,k,:] = (([cx0min_pxl_R,cy0min_pxl_R]))



In [ ]:
points_max_min_R.shape

### Remove outter circle duplicates

In [ ]:
#list for the final values
points_outter_R = []
for i in range(points_max_min_R.shape[0]):
    points_outter_R_list = []
    for j in range(points_max_min_R.shape[1]):
        points_deduplt_R = []
        points_max_min_dupl_R = points_max_min_R[i,j,:,:].tolist()

        [points_deduplt_R.append(item) for item in points_max_min_dupl_R if item not in points_deduplt_R]
        
        #print(np.shape(points_deduplt_R))
        points_outter_R_list.append(points_deduplt_R)
        #print(np.shape(points_outter_R_list))
    points_outter_R.append(points_outter_R_list)


In [ ]:
cx0_R.shape

## Inner circle corners

In [ ]:
points_inner_R = []
for i in range(cx0_R.shape[0]):
    points_inner_R_list = []
    for j in range(cx0_R.shape[1]):

        circ_R = np.sqrt((pixelGridXX-cx0_R[i,j,:])**2 + (pixelGridYY-cy0_R[i,j,:])**2) <= r0_R[i,j,:]
        circ_corners_R = np.where(circ_R == True)
        circ_corners_R = np.array(circ_corners_R)

        # transpose and obtain the values in mm
        circ_corners_arr_R = (circ_corners_R*pixel_width).T
        # swap the coordinates(for better readability)
        circ_corners_arr_R = np.array([circ_corners_arr_R[:,1],circ_corners_arr_R[:,0]]).T
        points_inner_R_list.append(circ_corners_arr_R.tolist())

    points_inner_R.append(points_inner_R_list)
#points_inner_L


In [ ]:
points_total_R = []
for i in range(cx0_R.shape[0]):
    points_total_R_list = []
    for j in range(cx0_R.shape[1]):
        points_total_R_list.append(points_inner_R[i][j]+points_outter_R[i][j])
    points_total_R.append(points_total_R_list)

### Remove outter and inner circle duplicates

In [ ]:
points_R = []
for i in range(np.shape(points_total_R)[0]):
    points_total_R_list = []
    for j in range(np.shape(points_total_R)[1]):
        points_R_list = []
        points_R_lst = points_total_R[i][j][:][:]

        [points_R_list.append(item) for item in points_R_lst if item not in points_R_list]

        points_total_R_list.append(points_R_list)
        
    points_R.append(points_total_R_list)

In [ ]:
points_R[0][0]

In [ ]:
points_R_arr = np.array(points_R[0][0])
np.shape(points_R_arr)

In [ ]:
cx0_R[0,2][0]

In [ ]:
cy0_R[0,0][1]

In [ ]:
np.shape(y0_L)

In [ ]:
points_R_arr = np.array(points_R[0][0])
for k in range(points_R_arr.shape[0]):
    print(pixel_cover_finder(gridSize,points_R_arr[:,0][k],points_R_arr[:,1][k],cx0_R[0,0][0],cy0_R[0,0][0],r0_R[0,0][0]))

In [ ]:
fig = plt.figure(figsize=(10,10))
fig.suptitle("Left and Right image coordinate pixelization")

centroid_L = np.zeros((cx0_L.shape[0],cx0_L.shape[1],2))
c0_L = np.zeros((cx0_L.shape[0],cx0_L.shape[1],2))

ax_L = fig.add_subplot()
#axis
ax_L.plot(pixelGridXX,pixelGridYY,'r')
ax_L.plot(pixelGridXX.T,pixelGridYY.T,'b')
ax_L.set_aspect(1)
ptchs_L =[]



# for the chessboards
for i in range(cx0_L.shape[0]):
    # for the circles
    for j in range(cx0_L.shape[1]):
        pixel_covered_percent_L = []
        points_L_arr = np.array(points_L[i][j])
        # for the squares that compose the circle
        for k in range(points_L_arr.shape[0]):
            #print(points_R_arr[:,0][k])
            #print(points_R_arr[:,1][k])
            #print(cx0_L[i,j,:])
            #print(cy0_L[i,j,:])
            #print(r0_L[i,j,:])
            pixel_covered_percent_L.append(pixel_cover_finder(gridSize,points_L_arr[:,0][k],points_L_arr[:,1][k],cx0_L[i,j][0],cy0_L[i,j][0],r0_L[i,j][0]))
            ptchs_L.append(Rectangle((points_L_arr[:,0][k],points_L_arr[:,1][k]), pixel_width,pixel_width))

            # center of the pixel
            ax_L.plot(cx0_L[i,j][0],cy0_L[i,j][0],'k.')
            ax_L.plot(x0_L[i,j],y0_L[i,j],'r',linewidth=4)

        #print(points_R_arr[:,0])
        pixel_covered_percent_array_L = np.array(pixel_covered_percent_L)
        #print(pixel_covered_percent_array_L)

        # centroid 
        centroid_L_x = np.sum(np.multiply(points_L_arr[:,0],pixel_covered_percent_array_L))/(np.sum(pixel_covered_percent_array_L)) + pixel_width/2
        centroid_L_y = np.sum(np.multiply(points_L_arr[:,1],pixel_covered_percent_array_L))/(np.sum(pixel_covered_percent_array_L)) + pixel_width/2

        #print(centroid_L_y)

        centroid_L[i,j,:] = ([centroid_L_x,centroid_L_y])
        c0_L[i,j,:] = ([cx0_L[i,j],cy0_L[i,j]])

        ax_L.plot(centroid_L_x,centroid_L_y,'y.')





    p_L = PatchCollection(ptchs_L, cmap=cm.jet, alpha=1)
    pixel_covered_percent_arr_L = np.array(pixel_covered_percent_L).flatten()
    # define the color
    p_L.set_array(pixel_covered_percent_arr_L)
    # set color max and min
    p_L.set_clim([0, 100])
    ax_L.add_collection(p_L)
    fig.colorbar(p_L)



            



In [ ]:
fig = plt.figure(figsize=(10,10))
fig.suptitle("Left and Right image coordinate pixelization")

centroid_R = np.zeros((cx0_R.shape[0],cx0_R.shape[1],2))
c0_R = np.zeros((cx0_R.shape[0],cx0_R.shape[1],2))

ax_R = fig.add_subplot()
#axis
ax_R.plot(pixelGridXX,pixelGridYY,'r')
ax_R.plot(pixelGridXX.T,pixelGridYY.T,'b')
ax_R.set_aspect(1)
ptchs_R =[]



# for the chessboards
for i in range(cx0_R.shape[0]):
    # for the circles
    for j in range(cx0_R.shape[1]):
        pixel_covered_percent_R = []
        points_R_arr = np.array(points_R[i][j])
        # for the squares that compose the circle
        for k in range(points_R_arr.shape[0]):
            #print(points_R_arr[:,0][k])
            #print(points_R_arr[:,1][k])
            #print(cx0_L[i,j,:])
            #print(cy0_L[i,j,:])
            #print(r0_L[i,j,:])
            pixel_covered_percent_R.append(pixel_cover_finder(gridSize,points_R_arr[:,0][k],points_R_arr[:,1][k],cx0_R[i,j][0],cy0_R[i,j][0],r0_R[i,j][0]))
            ptchs_R.append(Rectangle((points_R_arr[:,0][k],points_R_arr[:,1][k]), pixel_width,pixel_width))

            # center of the pixel
            ax_R.plot(cx0_R[i,j][0],cy0_R[i,j][0],'k.')
            ax_R.plot(x0_R[i,j],y0_R[i,j],'r',linewidth=4)

        #print(points_R_arr[:,0])
        pixel_covered_percent_array_R = np.array(pixel_covered_percent_R)
        #print(pixel_covered_percent_array_L)

        # centroid 
        centroid_R_x = np.sum(np.multiply(points_R_arr[:,0],pixel_covered_percent_array_R))/(np.sum(pixel_covered_percent_array_R)) + pixel_width/2
        centroid_R_y = np.sum(np.multiply(points_R_arr[:,1],pixel_covered_percent_array_R))/(np.sum(pixel_covered_percent_array_R)) + pixel_width/2

        #print(centroid_L_y)

        centroid_R[i,j,:] = ([centroid_R_x,centroid_R_y])
        c0_R[i,j,:] = ([cx0_R[i,j],cy0_R[i,j]])

        ax_R.plot(centroid_R_x,centroid_R_y,'y.')





    p_R = PatchCollection(ptchs_R, cmap=cm.jet, alpha=1)
    pixel_covered_percent_arr_R = np.array(pixel_covered_percent_R).flatten()
    # define the color
    p_R.set_array(pixel_covered_percent_arr_R)
    # set color max and min
    p_R.set_clim([0, 100])
    ax_R.add_collection(p_R)
    fig.colorbar(p_R)



In [ ]:
def repeat_func(xrange,yrange,zrange):

    translst= []

    for z in zrange:
        for x in xrange:
            for y in yrange:
                translst.append([x,y,z])
                
    len(translst)
    #print(translst)

    # chessboard print
    chess_pts = []
    world_pts = create_chessboard(chess_dimx,chess_dimy,chess_sq_size)
    rotangles = [[0,0,0]]

    for i in range(len(translst)):
        for j in range(len(rotangles)):    
            chess_pts.append(get_chessboard_rot_trans(world_pts,rotangles[j][0],rotangles[j][1],rotangles[j][2],translst[i][0],translst[i][1],translst[i][2]))

    chess_pts_arr = np.array(chess_pts)

    pattern_size_x,pattern_size_y = (chess_dimx,chess_dimy)

    X_pattern = np.linspace(0, pattern_size_x,pattern_size_x + 1)*chess_sq_size
    Y_pattern = np.linspace(0, pattern_size_y,pattern_size_y + 1)*chess_sq_size
    zdata = np.zeros((pattern_size_x + 1,pattern_size_y + 1))
    xdata, ydata = np.meshgrid(X_pattern, Y_pattern)

    xdata_ = xdata.flatten()
    ydata_ = ydata.flatten()
    zdata_ = zdata.flatten()
    
    # homogeneous coordinates
    world_pts =([xdata_,ydata_,zdata_])
    world_pts_ = np.array(world_pts).T

    world_pts_arr = np.zeros((chess_pts_arr.shape[0],(pattern_size_x+1)*(pattern_size_y+1),3),np.float32)
    
    for i in range(chess_pts_arr.shape[0]):
        world_pts_arr[i,:,:] = world_pts_

    world_pts_arr = world_pts_arr.reshape(world_pts_arr.shape[0],1,(pattern_size_x+1)*(pattern_size_y+1),3)
    
    x_lst_R = []
    x_lst_L = []
    for i in range(chess_pts_arr.shape[0]):
        x_arr_R, X_arr_R, E_R, K_R, P_R = get_image_points(chess_pts_arr[i,:,:],PX,PY,thetax= 0,thetay = 0,thetaz = 0,trans_x= -C_R[0],trans_y= -C_R[1],trans_z= -C_R[2],F = F,mx = (1/pixel_width),my =(1/pixel_width))
        x_arr_L, X_arr_L, E_L, K_L, P_L = get_image_points(chess_pts_arr[i,:,:],PX,PY,thetax= 0,thetay = 0,thetaz = 0,trans_x= -C_L[0],trans_y= -C_L[1],trans_z= -C_L[2],F = F,mx = (1/pixel_width),my =(1/pixel_width))
        
        x_lst_R.append(x_arr_R)
        x_lst_L.append(x_arr_L)

    x_zhang_R = np.array(x_lst_R,np.float32)
    x_zhang_L = np.array(x_lst_L,np.float32)

    print("World points: ",world_pts_arr.shape)
    print("Image points: ",x_zhang_L.shape)


    return world_pts_arr,x_zhang_R,x_zhang_L,P_R,P_L

In [ ]:
x_zhang_R = np.array((centroid_R/pixel_width),np.float32)
x_zhang_L = np.array((centroid_L/pixel_width),np.float32)

In [ ]:
ret_R_list = []
mtx_R_list = []
ret_L_list = []
mtx_L_list = []
retS_list = []
new_mtxL_list = []
new_mtxR_list = []
Rot_list = []
Trns_list = []
Emat_list = []
Fmat_list = []
P_R_org_list = []
P_L_org_list = []

world_pts_arr = 0
x_zhang_R = 0
x_zhang_L = 0

xrange_1 = np.linspace(-6000,6000,6)
yrange_1 = np.linspace(-4000,4000,6)
zrange_1 = ([20000])

world_pts_arr,x_zhang_R,x_zhang_L, P_R, P_L = repeat_func(xrange_1,yrange_1,zrange_1)
ret_R,mtx_R,ret_L,mtx_L,retS,new_mtxL,new_mtxR,Rot,Trns,Emat,Fmat = calib(world_pts_arr,x_zhang_R,x_zhang_L)

ret_R_list.append([ret_R])
mtx_R_list.append([mtx_R])
ret_L_list.append([ret_L])
mtx_L_list.append([mtx_L])
retS_list.append([retS])
new_mtxL_list.append([new_mtxL])
new_mtxR_list.append([new_mtxR])
Rot_list.append([Rot])
Trns_list.append([Trns])
Emat_list.append([Emat])
Fmat_list.append([Fmat])
P_R_org_list.append(P_R)
P_L_org_list.append(P_L)

In [ ]:
ret_R